# Exploratory Data Analysis

In [1]:
import pandas as pd
import numpy as np 
import json
import requests
import datetime
import matplotlib.pyplot as plt
import seaborn as sns
import ppscore
from scipy import stats

In [2]:
#load dataframe created in data wrangling phase
#parse datetime column
df= pd.read_csv('cleaned_cov_weather3.csv', parse_dates=['date'], index_col ='date')

In [4]:
df.head()

,vt_avg_temp,vt_prcp,me_avg_temp,me_prcp,ct_avg_temp,ct_prcp,ma_avg_temp,ma_prcp,CT_conf_cases,ME_conf_cases,MA_conf_cases,VT_conf_cases
date,,,,,,,,,,,,
2020-01-22,-28,0,-72.0,0.0,-76.0,0.0,-53.0,0.0,0,0,0,0
2020-01-23,-27,0,-30.0,3.0,-46.0,0.0,0.0,0.0,0,0,0,0
2020-01-24,-8,0,-14.0,0.0,6.0,0.0,19.0,0.0,0,0,0,0
2020-01-25,3,119,-30.0,3.0,23.0,218.0,33.0,25.0,0,0,0,0
2020-01-26,22,23,11.0,132.0,49.0,0.0,64.0,193.0,0,0,0,0


In [5]:
#double check data types
df.dtypes

vt_avg_temp        int64
vt_prcp            int64
me_avg_temp      float64
me_prcp          float64
ct_avg_temp      float64
ct_prcp          float64
ma_avg_temp      float64
ma_prcp          float64
CT_conf_cases      int64
ME_conf_cases      int64
MA_conf_cases      int64
VT_conf_cases      int64
dtype: object

In [6]:
#Convert int columns to float in preparation for calculations:
int_columns= {"CT_conf_cases": 'float', "ME_conf_cases": 'float', "MA_conf_cases":"float", "VT_conf_cases":'float', "vt_avg_temp":'float', "vt_prcp":'float'}
df = df.astype(int_columns)

In [7]:
df.dtypes

vt_avg_temp      float64
vt_prcp          float64
me_avg_temp      float64
me_prcp          float64
ct_avg_temp      float64
ct_prcp          float64
ma_avg_temp      float64
ma_prcp          float64
CT_conf_cases    float64
ME_conf_cases    float64
MA_conf_cases    float64
VT_conf_cases    float64
dtype: object

In [8]:
#Any missing values?
df[pd.isnull(df).any(axis=1)]

,vt_avg_temp,vt_prcp,me_avg_temp,me_prcp,ct_avg_temp,ct_prcp,ma_avg_temp,ma_prcp,CT_conf_cases,ME_conf_cases,MA_conf_cases,VT_conf_cases
date,,,,,,,,,,,,
2020-09-07,194.0,0.0,NaN,NaN,217.0,0.0,222.0,0.0,53365.0,4701.0,122791.0,1651.0


No missing values.

However, the units of TAVG and PRCP are clearly off. 

According to [NOAA documentation](https://www1.ncdc.noaa.gov/pub/data/ghcn/daily/readme.txt), temperatures are reported in __*tenths*__ of a degree Celsius. I'll convert to degrees Fahrenheit instead (since this is US data here).

In [ ]:
# (C*9/50)+32=F (since C is reported here as tenths of degrees)
df2=df
df2['vt_avg_temp'] = ((df['vt_avg_temp']*9)/50) + 32
df2['me_avg_temp'] = ((df['me_avg_temp']*9)/50) + 32
df2['ct_avg_temp'] = ((df['ct_avg_temp']*9)/50) + 32
df2['ma_avg_temp'] = ((df['ma_avg_temp']*9)/50) + 32

The [same documention](https://www1.ncdc.noaa.gov/pub/data/ghcn/daily/readme.txt) cited above also explains that PRCP data is reported in __*tenths*__ of a mm. [Wikipedia](https://en.wikipedia.org/wiki/Precipitation) reports that generally PRCP data is reported in millimeters, so I'll convert to millimeters instead.

In [ ]:
df3=df2
df3['vt_prcp'] = df2['vt_prcp']/10
df3['me_prcp'] = df2['me_prcp']/10
df3['ct_prcp'] = df2['ct_prcp']/10
df3['ma_prcp'] = df2['ma_prcp']/10

In [ ]:
df3.head()

For the sake of clarity, I'll update column names to reflect this unit change.

In [ ]:
#Update column names to reflect new units:
units_dict= {'vt_avg_temp': 'VT_Avg_Temp(F)', 'vt_prcp': 'VT_PRCP(mm)', 'me_avg_temp':'ME_Avg_Temp(F)', 'me_prcp': 'ME_PRCP(mm)', 'ct_avg_temp': 'CT_Avg_Temp(F)', 'ct_prcp':'CT_PRCP(mm)', 'ma_avg_temp': 'MA_Avg_Temp(F)', 'ma_prcp':'MA_PRCP(mm)'}
df3.rename(columns=units_dict, inplace=True)
df3.head()

In [ ]:
#Examine timespan of data:
print('Beginning date: ',min(df3.index))
print('End date: ', max(df3.index))
print('Total timespan: ', max(df3.index-min(df3.index)))

In [ ]:
# Make copy of df 
df4=df3

In [ ]:
# Make column of non-cumulative counts of confirmed cases for each state:
df4["CT_Conf_Cases"] = df4['CT_conf_cases'].diff().fillna(0)
df4["VT_Conf_Cases"] = df4['VT_conf_cases'].diff().fillna(0)
df4["ME_Conf_Cases"] = df4['ME_conf_cases'].diff().fillna(0)
df4["MA_Conf_Cases"] = df4['MA_conf_cases'].diff().fillna(0)

In [ ]:
#Drop cumulative confirmed case columns:
df5 = df4.drop(columns=['CT_conf_cases', 'VT_conf_cases', 'ME_conf_cases', 'MA_conf_cases'])

In [ ]:
#Reset DateTimeIndex
df5.set_index('date',inplace=True)
df5

In [ ]:
#Take a look at some summary statistics of the data:
df5.describe()

It looks like some of the states have `Conf_Cases` values that are less than zero (see `min` row). That can't be right; I'll replace them with zeroes instead.

In [ ]:
#Replace negative values with zeroes:
df5[df5 < 0] = 0

A quick look at the tail of the DataFrame reveals another interesting detail:

In [ ]:
df5['CT_Conf_Cases'].tail(25)

It looks like, for every five days of values for `Conf_Cases`, there are two days of zero values. A quick calendar check confirms that the dates corresponding to zero values are weekend dates (besides 09/06/21, which was Labor Day- a bank holiday in the US). Because there is only data for weekdays, I'll remove weekend data, as it can only skew our summary statistics and plots.

In [ ]:
#Create column to designate day of week:
df6= df5
df6["weekday"] = df6.index.dayofweek

In [ ]:
#Filter DataFrame to only include weekdays (days 0-4)
df6 = df6[(df6.weekday != 5) & (df6.weekday != 6)]

Another quick check of the summary statistics:

In [ ]:
df6.describe()

In [ ]:
df6.head()

In [ ]:
#Create a list of US bank holidays in 2020-2021:
#Please refer to note below, explaining how this list of dates was put together.
US_bank_holidays = ['2020-02-17', '2020-12-25', '2020-11-26', '2020-12-25', '2021-01-01', '2021-05-31', '2021-09-06']

Full list of 2020 US Holidays [here](https://www.officeholidays.com/countries/usa/2020).
Full list of 2021 US Holidays [here](https://www.officeholidays.com/countries/usa/2021).\
__Note:__ Not all bank holidays listed in the above resources appear in my `US_bank_holidays` list. I used the resources above as a general guide of dates that might not contain any data, and then checked each of the dates individually to make sure I wasn't deleting any significant data from the set. I found that data __was__ recorded for some of the bank holidays listed in the resources; whenever this was the case, I did __not__ drop that row of data. The dates listed in `US_bank_holidays` reflect only those bank holidays listed in these resources that I was __also__ able to confirm did not have any significant data.

In [ ]:
#Drop rows with indices in US_bank_holidays list:
df7= df6.drop(df6[(df6.index.isin(US_bank_holidays))].index)

In [ ]:
#Check rows have been dropped:
print(len(df6.index))
print(len(df7.index))

In [ ]:
#Recheck summary statistics with dropped US_bank_holidays:
df7.describe()

## Plots:

With so many twin axes to plot, I'll define a function to help plot all of twin axes time series data:

In [ ]:
def plot_timeseries(axes, x, y, color, xlabel, ylabel):
    axes.plot(x,y,color=color)
    axes.set_xlabel(xlabel)
    axes.set_ylabel(ylabel)
    axes.tick_params('y', colors = color)

## Connecticut

In [ ]:
#Plot CT time vs confirmed cases
fig, ax = plt.subplots()
plot_timeseries(ax, df7.index, df7['CT_Conf_Cases'], 'blue', 'Time', 'Confirmed Cases')
plt.xticks(rotation=90)
ax2 = ax.twinx()
#Plot CT temp vs time
plot_timeseries(ax2, df7.index, df7['CT_Avg_Temp(F)'], 'red', 'Time', 'Average Temperature (F)')
plt.title('Connecticut temp vs conf cases over time')
plt.xticks(rotation=90)
plt.show()

In [ ]:
#Plot CT time vs confirmed cases
fig, ax = plt.subplots()
plot_timeseries(ax, df7.index, df7['CT_Conf_Cases'], 'blue', 'Time', 'Confirmed Cases')
plt.xticks(rotation=90)
ax2 = ax.twinx()
#Plot CT prcp vs time
plot_timeseries(ax2, df7.index, df7['CT_PRCP(mm)'], 'red', 'Time', 'CT_PRCP(mm)')
plt.title('Connecticut prcp vs confirmed cases over time')
plt.xticks(rotation=90)
plt.show()

***

## Maine

In [ ]:
#Plot ME time vs Avg. Temp 
fig, ax = plt.subplots()
plot_timeseries(ax, df7.index, df7['ME_Conf_Cases'], 'blue', 'Time', 'Confirmed Cases')
plt.xticks(rotation=90)
ax2 = ax.twinx()
plot_timeseries(ax2, df7.index, df7['ME_Avg_Temp(F)'], 'red', 'Time', 'Average Temperature (F)')
plt.title('Maine Temp vs Conf Cases')
plt.xticks(rotation=90)
plt.show()

In [ ]:
#Plot ME time vs confirmed cases
fig, ax = plt.subplots()
plot_timeseries(ax, df7.index, df7['ME_Conf_Cases'], 'blue', 'Time', 'Confirmed Cases')
plt.xticks(rotation=90)
ax2 = ax.twinx()
#Plot ME prcp vs time
plot_timeseries(ax2, df7.index, df7['ME_PRCP(mm)'], 'red', 'Time', 'ME_PRCP(mm)')
plt.title('Maine prcp vs confirmed cases over time')
plt.xticks(rotation=90)
plt.show()

## Massachusetts

In [ ]:
#Plot MA time vs confirmed cases
fig, ax = plt.subplots()
plot_timeseries(ax, df7.index, df7['MA_Conf_Cases'], 'blue', 'Time', 'Confirmed Cases')
plt.xticks(rotation=90)
ax2 = ax.twinx()
#Plot MA temp vs time
plot_timeseries(ax2, df7.index, df7['MA_Avg_Temp(F)'], 'red', 'Time', 'MA_Avg_Temp(F)')
plt.title('Massachusetts TAVG vs confirmed cases over time')
plt.xticks(rotation=90)
plt.show()

In [ ]:
#Plot MA time vs confirmed cases
fig, ax = plt.subplots()
plot_timeseries(ax, df7.index, df7['MA_Conf_Cases'], 'blue', 'Time', 'Confirmed Cases')
plt.xticks(rotation=90)
ax2 = ax.twinx()
#Plot MA prcp vs time
plot_timeseries(ax2, df7.index, df7['MA_PRCP(mm)'], 'red', 'Time', 'MA_PRCP(mm)')
plt.title('Massachusetts prcp vs confirmed cases over time')
plt.xticks(rotation=90)
plt.show()

## Vermont

In [ ]:
#Plot VT time vs confirmed cases
fig, ax = plt.subplots()
plot_timeseries(ax, df7.index, df7['VT_Conf_Cases'], 'blue', 'Time', 'Confirmed Cases')
plt.xticks(rotation=90)
ax2 = ax.twinx()
#Plot VT temp vs time
plot_timeseries(ax2, df7.index, df7['VT_Avg_Temp(F)'], 'red', 'Time', 'VT_Avg_Temp(F)')
plt.title('Vermont TAVG vs confirmed cases over time')
plt.xticks(rotation=90)
plt.show()

In [ ]:
#Plot VT time vs confirmed cases
fig, ax = plt.subplots()
plot_timeseries(ax, df7.index, df7['VT_Conf_Cases'], 'blue', 'Time', 'Confirmed Cases')
plt.xticks(rotation=90)
ax2 = ax.twinx()
#Plot MA prcp vs time
plot_timeseries(ax2, df7.index, df7['VT_PRCP(mm)'], 'red', 'Time', 'VT_PRCP(mm)')
plt.title('Vermont prcp vs confirmed cases over time')
plt.xticks(rotation=90)
plt.show()

## Perform Shapiro-Wilk test for normality:

In [ ]:
#Convert dataframe to array to perform Shapiro-Wilcox test:
np_cov_weather7 = df7.to_numpy()

In [ ]:
#Apply Shapiro test:
shapiro_test = stats.shapiro(np_cov_weather7)

In [ ]:
#Print values:
print("Shapiro test statistic is: ", shapiro_test.statistic)
print("Shapiro p-value is: ", shapiro_test.pvalue)

## Predictive Power Score:

In [ ]:
#Calculate ppscore 
pps = ppscore.matrix(df7)

In [ ]:
print(pps)

In [ ]:
print(df7.dtypes)

In [ ]:
matrix_df = ppscore.matrix(df7).pivot(columns='x', index='y',  values='ppscore')

In [ ]:
sns.set(rc={'figure.figsize':(11.7,8.27)})
sns.heatmap(matrix_df, annot= True)

In [ ]:

sns.set(rc={'figure.figsize':(11.7,8.27)})
sns.heatmap(df7.corr(), square=True, cmap='rocket')

In [ ]:
Hartford_CT = df7[['CT_Avg_Temp(F)', 'CT_PRCP(mm)', 'CT_Conf_Cases']]

In [ ]:
sns.heatmap(Hartford_CT.corr(), square=True, cmap='rocket')

In [ ]:
Middlesex_MA = df7[['MA_Avg_Temp(F)', 'MA_PRCP(mm)', 'MA_Conf_Cases']]

In [ ]:
sns.heatmap(Middlesex_MA.corr(), square=True, cmap='rocket')

In [ ]:
Essex_VT = df7[['VT_Avg_Temp(F)', 'VT_PRCP(mm)', 'VT_Conf_Cases']]

In [ ]:
sns.heatmap(Essex_VT.corr(), square=True, cmap='rocket')

In [ ]:
Piscataquis_ME = df7[['ME_Avg_Temp(F)', 'ME_PRCP(mm)', 'ME_Conf_Cases']]

In [ ]:
sns.heatmap(Piscataquis_ME.corr(), square=True, cmap='rocket')

AWND
GHCND:USW00054704 MA
    
GHCND:USW00014740 CT

GHCND:USW00014742 VT

GHCND:USW00094626 ME